# Trabajo Práctico Clasificación

## Integrantes:

* Augusto Kark
* Lucas Garcia

## Descripción del trabajo:

> Consiste en el desarrollo de un modelo de clasificación para el diagnóstico de cáncer de seno maligno utilizando el dataset de sklearn (breast_cancer).

Se implementara lo visto en clase paso por paso junto a una descripción detallada.


---
## Importamos librerias y creamos el dataframe

> Se agrega la columna 'diagnostico' que contiene el resultado del diagnóstico (0: maligno, 1: benigno)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer


data = load_breast_cancer()

df = pd.DataFrame(data=data.data, columns=data.feature_names)

df["diagnostico"] = data.target

---

## Limpiamos y entendemos el dataset

> El DataFrame contiene 569 entradas y 31 columnas. Cada columna representa una característica del conjunto de datos de cáncer de mama, con 30 columnas de tipo float64 y una columna de tipo int64 que representa el diagnóstico. No hay valores nulos en ninguna de las columnas. El conjunto de datos no contiene valores duplicados.

In [ ]:
# Entendemos el dataset
df.info()
df.describe()
print(data.target_names)
print(data.target)

# Revisamos si hay valores duplicados
print(f"Cantidad de datos duplicados: {df.duplicated().sum()}")

---
## Dividimos el dataset

> Utilizamos StratifiedKFold para dividir el conjunto de datos en 5 pliegues, manteniendo la proporción de clases en cada grupo de entrenamiento y prueba. Escalamos los datos con StandardScaler para normalizarlos. Implementamos una función de visualización para mostrar la partición de los conjuntos de datos, utilizando matplotlib y ListedColormap para colorear las clases y los grupos de entrenamiento y prueba. Calculamos y mostramos la proporción de clases en cada pliegue de entrenamiento y prueba.

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from matplotlib.patches import Patch
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

cmap_clases = ListedColormap(['#f21b3f', '#ff9914'])
cmap_train_test = ListedColormap(['#8ac926', '#1982c4'])

def visualize_split(classes, groups, splits):
    fig, ax = plt.subplots(figsize=(10, 6))
    espaciado = list(range(splits, 0, -1))

    # Graficar las clases
    ax.scatter(range(len(classes)), [splits + 1.0] * len(classes), c=classes, marker='_', lw=25, cmap=cmap_clases)
    legend1 = ax.legend([Patch(color=cmap_clases(0.0)), Patch(color=cmap_clases(1.0))], ['Clase 0', 'Clase 1'], loc='upper right')

    if splits == 1:
        ax.scatter(range(len(groups)), [splits] * len(groups), c=groups, marker='_', lw=25, cmap=cmap_train_test)
        ax.set(yticks=[splits, splits + 1.0], yticklabels=['Entrenamiento/Testeo', 'Clases'], xlabel="Observaciones")
        ax.legend([Patch(color=cmap_train_test(0.0)), Patch(color=cmap_train_test(1.0))], ['Entrenamiento', 'Testeo'], loc='lower right')
        ax.add_artist(legend1)

    elif splits == len(classes):
        for i in range(len(espaciado)):
            ax.scatter(range(len(groups[i])), [espaciado[i]] * len(groups[i]), c=groups[i], marker='_', lw=25, cmap=cmap_train_test)
        ax.legend([Patch(color=cmap_train_test(0.0)), Patch(color=cmap_train_test(1.0))], ['Entrenamiento', 'Testeo'], loc='lower right')
        ax.add_artist(legend1)
        if splits < 11:
            ticklabels = [f'CV {x}' for x in reversed(espaciado)]
            ticklabels.append('Clases')
            espaciado.append(splits + 1)
            ax.set(yticks=espaciado, yticklabels=ticklabels, xlabel="Observaciones")

    else:
        for i in range(len(espaciado)):
            ax.scatter(range(len(groups[i])), [espaciado[i]] * len(groups[i]), c=groups[i], marker='_', lw=25, cmap=cmap_train_test)
        ax.legend([Patch(color=cmap_train_test(0.0)), Patch(color=cmap_train_test(1.0))], ['Entrenamiento', 'Testeo'], loc='lower right')
        ax.add_artist(legend1)
        if splits < 11:
            ticklabels = [f'CV {x}' for x in reversed(espaciado)]
            ticklabels.append('Clases')
            espaciado.append(splits + 1)
            ax.set(ylim=[0, splits + 2], yticks=espaciado, yticklabels=ticklabels, xlabel="Observaciones")

    plt.title('Visualización de la Partición de Conjuntos')
    plt.show()

X = df.drop("diagnostico", axis=1)
y = df["diagnostico"]

# Escalamos los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

n_splits = 5

clases = y

skf = StratifiedKFold(n_splits=n_splits)

grupos = []
folds_indices = []

# Datos de la partición
nro = 1
for train_index, test_index in skf.split(X_scaled, y):
    grupo = np.zeros(y.shape)
    for i in range(len(grupo)):
        if i in train_index:
            grupo[i] = 0
        else:
            grupo[i] = 1

    grupos.append(grupo)
    
    folds_indices.append((train_index, test_index))

    train_C0 = 0
    train_C1 = 0
    test_C0 = 0
    test_C1 = 0

    for j in range(len(grupo)):  # recorro la información de grupo de entrenamiento/test
        if grupo[j] == 0 and clases[j] == 0:
            train_C0 += 1
        if grupo[j] == 0 and clases[j] == 1:
            train_C1 += 1
        if grupo[j] == 1 and clases[j] == 0:
            test_C0 += 1
        if grupo[j] == 1 and clases[j] == 1:
            test_C1 += 1

    print('CV %d: Entrenamiento %.2f C1/C0 | Test %.2f C1/C0' % (nro, train_C1 / train_C0, test_C1 / test_C0))
    nro += 1

visualize_split(clases, grupos, n_splits)

---
## Ajuste

> Aplicamos PCA para reducir los datos a 2 componentes principales y creamos un gráfico de dispersión (scatter plot) para visualizar la proyección PCA de las clases en el conjunto de datos de cáncer de mama, diferenciando entre las clases malignas y benignas con diferentes colores. Se logra ver una separación lineal entre las clases malignas y benignas por lo que utilizamos un Análisis Discriminante Lineal para el ajuste.

In [ ]:
# Importar las bibliotecas necesarias
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA

# Aplicar PCA para reducir los datos a 2 componentes principales
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Crear un gráfico de dispersión (scatter plot)
plt.figure(figsize=(8, 6))
colors = ['red', 'blue']
labels = ['Malignant', 'Benign']

for i in range(2):
    plt.scatter(X_pca[y == i, 0], X_pca[y == i, 1], c=colors[i], label=labels[i], alpha=0.5)

# Añadir etiquetas y leyenda
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.title('Proyección PCA de las clases en el conjunto de datos de cáncer de mama')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Definir el modelo LDA
lda = LinearDiscriminantAnalysis()

# Lista para almacenar los modelos entrenados
trained_models = []

# Entrenar el modelo en cada fold
for train_index, test_index in folds_indices:
    # Dividir los datos en entrenamiento y prueba usando los índices generados
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Entrenar el modelo LDA
    lda.fit(X_train, y_train)

    # Almacenar el modelo entrenado
    trained_models.append((lda, test_index))

---
##  Visualización de fronteras de decisión y métricas

> Evaluamos el rendimiento del modelo LDA en cada conjunto de prueba y calculamos las métricas de precisión, precisión, recall y F1-score. Además, visualizamos las fronteras de decisión utilizando PCA para reducir los datos a 2 componentes principales y creamos gráficos de dispersión para cada conjunto de prueba. Finalmente, mostramos las métricas promedio y la matriz de confusión acumulada.

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn import metrics
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import seaborn as sns

# Set the style for prettier plots
plt.style.use('seaborn-v0_8-pastel')
sns.set_palette("deep")

# Lista para almacenar las métricas
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
confusion_matrices = []

# Evaluar cada modelo en sus respectivos conjuntos de prueba
for i, (model, test_index) in enumerate(trained_models):
    # Obtener los datos de prueba correspondientes
    X_test = X_scaled[test_index]
    y_test = y[test_index]

    # Hacer predicciones
    y_pred = model.predict(X_test)

    # Calcular las métricas y almacenarlas
    accuracy_scores.append(accuracy_score(y_test, y_pred))
    precision_scores.append(precision_score(y_test, y_pred))
    recall_scores.append(recall_score(y_test, y_pred))
    f1_scores.append(f1_score(y_test, y_pred))
    confusion_matrices.append(confusion_matrix(y_test, y_pred))

    # Visualización de fronteras de decisión
    pca = PCA(n_components=2)
    X_test_pca = pca.fit_transform(X_test)
    xx, yy = np.meshgrid(np.linspace(X_test_pca[:, 0].min(), X_test_pca[:, 0].max(), 1000),
                         np.linspace(X_test_pca[:, 1].min(), X_test_pca[:, 1].max(), 1000))
    Z = model.predict(pca.inverse_transform(np.c_[xx.ravel(), yy.ravel()]))
    Z = Z.reshape(xx.shape)

    plt.figure(figsize=(6, 4))
    plt.contourf(xx, yy, Z, alpha=0.8, cmap='RdYlBu')
    scatter = plt.scatter(X_test_pca[:, 0], X_test_pca[:, 1], c=y_test, edgecolor='black', cmap='RdYlBu', s=50)
    plt.xlabel('Principal Component 1', fontsize=12)
    plt.ylabel('Principal Component 2', fontsize=12)
    plt.title('LDA Decision Boundary (Breast Cancer Dataset)', fontsize=16)
    plt.colorbar(scatter)
    plt.tight_layout()
    plt.show()

    # Visualización 3D de LDA
    xx, yy = np.meshgrid(np.linspace(X_test_pca[:, 0].min(), X_test_pca[:, 0].max(), 100),
                         np.linspace(X_test_pca[:, 1].min(), X_test_pca[:, 1].max(), 100))

    Z = model.predict_proba(pca.inverse_transform(np.c_[xx.ravel(), yy.ravel()]))[:, 1]
    Z = Z.reshape(xx.shape)

    # 3D Plot with improvements
    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(111, projection='3d')

    # Smoother surface, improved colormap, and transparency
    surf = ax.plot_surface(xx, yy, Z, cmap='coolwarm', alpha=0.7, antialiased=True, rstride=1, cstride=1,
                           edgecolor='none', shade=True)

    # Scatter plot for data points with better edge contrast
    scatter = ax.scatter(X_test_pca[:, 0], X_test_pca[:, 1], y_test, c=y_test, cmap='coolwarm',
                         edgecolor='black', s=60, alpha=1)

    # Axis labels
    ax.set_xlabel('Principal Component 1', fontsize=12)
    ax.set_ylabel('Principal Component 2', fontsize=12)
    ax.set_zlabel('Probability', fontsize=12)
    ax.set_title('3D Visualization of LDA', fontsize=16)

    # Improved viewpoint (adjust `elev` and `azim` as needed)
    ax.view_init(elev=30, azim=240)

    # Color bar for surface plot
    fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)

    # Show the plot
    plt.tight_layout()
    plt.show()

# Mostrar las métricas
print("\nMétricas de rendimiento del modelo LDA:")
print(f'Precisión media: {np.mean(accuracy_scores):.4f} (±{np.std(accuracy_scores):.4f})')
print(f'Precisión media: {np.mean(precision_scores):.4f} (±{np.std(precision_scores):.4f})')
print(f'Recall medio: {np.mean(recall_scores):.4f} (±{np.std(recall_scores):.4f})')
print(f'F1-score medio: {np.mean(f1_scores):.4f} (±{np.std(f1_scores):.4f})')

# Visualizar la matriz de confusión acumulada
plt.figure(figsize=(5, 3))
sns.heatmap(np.sum(confusion_matrices, axis=0), annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Accumulated Confusion Matrix')
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

---
## Ajuste de hiperparámetros

> En esta sección, realizamos un ajuste de hiperparámetros para el modelo de Análisis Discriminante Lineal (LDA) utilizando RandomizedSearchCV. Se probaron diferentes combinaciones de hiperparámetros para encontrar la configuración óptima que maximice la precisión del modelo. A continuación, se presentan las estadísticas de los parámetros obtenidos y los mejores parámetros globales seleccionados:

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint
import numpy as np
from collections import defaultdict

# Definir los hiperparámetros a probar
param_distributions_list = [
    {
        'solver': ['eigen', 'lsqr'],
        'shrinkage': uniform(0, 1),
        'n_components': randint(1, min(len(np.unique(y)) - 1, X_scaled.shape[1]) + 1),
        'priors': [None],
        'store_covariance': [False, True],
        'covariance_estimator': [None]
    },
    {
        'solver': ['svd'],
        'tol': uniform(1e-5, 1e-3),
        'n_components': randint(1, min(len(np.unique(y)) - 1, X_scaled.shape[1]) + 1),
        'priors': [None],
        'store_covariance': [False, True]
    }
]

# Diccionario para almacenar todos los valores de parámetros
all_params = defaultdict(list)

# Entrenar el modelo en cada fold
for train_index, test_index in folds_indices:
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]

    for param_distributions in param_distributions_list:
        random_search = RandomizedSearchCV(
            LinearDiscriminantAnalysis(),
            param_distributions=param_distributions,
            n_iter=100,
            cv=5,
            scoring='accuracy',
            n_jobs=-1,
            random_state=42
        )
        random_search.fit(X_train, y_train)

        # Almacenar los mejores parámetros
        for key, value in random_search.best_params_.items():
            all_params[key].append(value)

# Calcular y mostrar estadísticas de los parámetros
print("\nEstadísticas de los parámetros:")
for key, values in all_params.items():
    if isinstance(values[0], (int, float)):
        mean_value = np.mean(values)
        median_value = np.median(values)
        std_value = np.std(values)
        print(f"{key}:")
        print(f"  Media: {mean_value:.6f}")
        print(f"  Mediana: {median_value:.6f}")
        print(f"  Desviación estándar: {std_value:.6f}")
    else:
        value_counts = defaultdict(int)
        for value in values:
            value_counts[value] += 1
        most_common = max(value_counts, key=value_counts.get)
        print(f"{key}:")
        print(f"  Valor más común: {most_common}")
        print(f"  Distribución: {dict(value_counts)}")
    print()

# Calcular los mejores parámetros globales
best_params = {}
for key, values in all_params.items():
    if isinstance(values[0], (int, float)):
        best_params[key] = np.median(values)
    else:
        best_params[key] = max(set(values), key=values.count)

print("Mejores parámetros globales:")
for key, value in best_params.items():
    print(f"{key}: {value}")

---

## Nuevos Parámetros

> Utilizamos los mejores parámetros obtenidos durante el ajuste de hiperparámetros para entrenar un nuevo modelo de Análisis Discriminante Lineal (LDA) y compararlo con el modelo anterior. Notamos que no hay una mejora significativa.

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Set the style for prettier plots
plt.style.use('seaborn-v0_8-pastel')
sns.set_palette("deep")

# Function to evaluate the model
def evaluate_model(trained_models, X_scaled, y):
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []
    confusion_matrices = []

    for i, (model, test_index) in enumerate(trained_models):
        X_test = X_scaled[test_index]
        y_test = y[test_index]
        y_pred = model.predict(X_test)

        accuracy_scores.append(accuracy_score(y_test, y_pred))
        precision_scores.append(precision_score(y_test, y_pred))
        recall_scores.append(recall_score(y_test, y_pred))
        f1_scores.append(f1_score(y_test, y_pred))
        confusion_matrices.append(confusion_matrix(y_test, y_pred))

    print("\nMétricas de rendimiento del modelo LDA:")
    print(f'Precisión media: {np.mean(accuracy_scores):.4f} (±{np.std(accuracy_scores):.4f})')
    print(f'Precisión media: {np.mean(precision_scores):.4f} (±{np.std(precision_scores):.4f})')
    print(f'Recall medio: {np.mean(recall_scores):.4f} (±{np.std(recall_scores):.4f})')
    print(f'F1-score medio: {np.mean(f1_scores):.4f} (±{np.std(f1_scores):.4f})')

    plt.figure(figsize=(5, 3))
    sns.heatmap(np.sum(confusion_matrices, axis=0), annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Accumulated Confusion Matrix')
    plt.tight_layout()
    plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
    plt.show()

# Train and evaluate the model without hyperparameter optimization
lda_default = LinearDiscriminantAnalysis()
trained_models_default = []

for train_index, test_index in folds_indices:
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lda_default.fit(X_train, y_train)
    trained_models_default.append((lda_default, test_index))

print("Evaluación sin optimización de hiperparámetros:")
evaluate_model(trained_models_default, X_scaled, y)

# Train and evaluate the model with hyperparameter optimization
lda_optimized = LinearDiscriminantAnalysis(
    solver=best_params.get('solver'),
    n_components=int(best_params.get('n_components')),
    priors=best_params.get('priors'),
    store_covariance=bool(best_params.get('store_covariance')),
    covariance_estimator=best_params.get('covariance_estimator')
)
trained_models_optimized = []

for train_index, test_index in folds_indices:
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lda_optimized.fit(X_train, y_train)
    trained_models_optimized.append((lda_optimized, test_index))

print("Evaluación con optimización de hiperparámetros:")
evaluate_model(trained_models_optimized, X_scaled, y)